# Sumário

[Funções de Ativação](#Funções-de-Ativação)

[Implementação](#Implementação)

[Teste](#Teste)

[Referências](#Referências)

# Imports and Configurações

In [1]:
import numpy as np

# Funções de Ativação

In [2]:
def linear(x, derivative=False):
    return np.ones_like(x) if derivative else x

def sigmoid(x, derivative=False):
    if derivative:
        y = sigmoid(x)
        return y*(1-y)
    return 1.0/(1.0 + np.exp(-x))

def tanh(x, derivative=False):
    if derivative:
        y = tanh(x)
        return 1 - y**2
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

def relu(x, derivative=False):
    if derivative:
        return np.where(x <= 0, 0, 1)
    return np.maximum(0, x)

def leaky_relu(x, derivative=False):
    if derivative:
        return np.where(x <= 0, 0.1, 1)
    return np.where(x < 0, 0.1*x, x)

def gaussian(x, derivative=False):
    if derivative:
        return -2*x*np.exp(-x**2)
    return np.exp(-x**2)

# Funções de custo

In [3]:
def mse(y, y_pred, derivative=False):
    if derivative:
        return np.mean(-(y - y_pred))
    return np.mean((y - y_pred)**2)

def sigmoid_cross_entropy(y, y_pred, derivative=False):
    if derivative:
        pass
    return -np.mean(y*np.log(y_pred) + (1-y)*np.log(1-y_pred))

def softmax_cross_entropy(y, y_pred, derivative=False):
    if derivative:
        return -( y*(1/y_pred) + (1-y)*(1/(1-y_pred)) )
    return -np.mean((y*np.log(y_pred) + (1-y)*np.log(1-y_pred)).sum(axis=1))

# Implementação 

In [4]:
class NeuralNetwork():
    def __init__(self, layers_size, activations, cost_func, learning_rate=1e-3):
        self.layers_size = layers_size
        self.activations = activations
        self.cost_func = cost_func
        self.learning_rate = learning_rate
        self._activ_inp = []
        self._activ_out = []
        self.weights = [np.random.randn(out, inp) for inp, out in zip(self.layers_size[:-1], self.layers_size[1:])]
        self.biases = [np.random.randn(1, out) for out in self.layers_size[1:]]
            
    def fit(self, x, y, epochs=100, verbose=10):
        for epoch in range(epochs):
            y_pred = self.__feedforward(x)
            self.__backprop(y, y_pred)
            
            if epoch % verbose == 0:
                cost = self.cost_func(y, y_pred)
                print("epoch: {0:=4}/{1} cost: {2}".format(epoch, epochs, cost))
    
    def predict(self, x):
        return self.__feedforward(x)
    
    def __feedforward(self, x):
        self._activ_inp, self._activ_out = [], []
        self._activ_out.append(x)
        for w, b, activation in zip(self.weights, self.biases, self.activations):
            y = np.dot(self._activ_out[-1], w.T) + b
            self._activ_inp.append(y)
            self._activ_out.append(activation(y))
        return self._activ_out[-1]
    
    def __backprop(self, y, y_pred):
        self._activ_out.pop()
        
        last_delta = self.cost_func(y, y_pred, derivative=True)
        dweights, dbiases = [], []
        for inp, out, w, activation in zip(reversed(self._activ_inp), reversed(self._activ_out), reversed(self.weights), reversed(self.activations)):
            dactivation = activation(inp, derivative=True)*last_delta
            last_delta = np.dot(dactivation, w)
            dweights.append(np.dot(dactivation.T, out))
            dbiases.append(1.0*dactivation.sum(axis=0, keepdims=True))
        
        self.weights = [w - self.learning_rate*dw for w, dw in zip(self.weights, reversed(dweights))]
        self.biases  = [b - self.learning_rate*db for b, db in zip(self.biases, reversed(dbiases))]

# Teste

In [5]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0]).reshape(-1, 1)

print(x.shape, y.shape)

(4, 2) (4, 1)


In [9]:
D = x.shape[1]
nn = NeuralNetwork(layers_size=[D, 3, 4, 3, 1], activations=[relu, relu, relu, linear], cost_func=mse, learning_rate=1e-2)

print([w.T.shape for w in nn.weights])
print([b.shape for b in nn.biases])
nn.fit(x, y)
nn.predict(x)

[(2, 3), (3, 4), (4, 3), (3, 1)]
[(1, 3), (1, 4), (1, 3), (1, 1)]
epoch:    0/100 cost: 5.600268569064188
epoch:   10/100 cost: 0.111840369854695
epoch:   20/100 cost: 0.11331317892708881
epoch:   30/100 cost: 0.11341671000146131
epoch:   40/100 cost: 0.11341886335130982
epoch:   50/100 cost: 0.1134189074967369
epoch:   60/100 cost: 0.11341890840148823
epoch:   70/100 cost: 0.11341890842003076
epoch:   80/100 cost: 0.11341890842041087
epoch:   90/100 cost: 0.11341890842041856


array([[ 0.39883793],
       [ 0.73957958],
       [ 1.26034052],
       [-0.39875802]])

# Referências